<a href="https://colab.research.google.com/github/MatthewBaggins/fastbook/blob/master/PAC_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 5.1MB/s 
     |████████████████████████████████| 1.0MB 26.4MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 194kB 47.9MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
     |████████████████████████████████| 40kB 4.7MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 2.6MB 31.9MB/s 
Mounted at /content/gdrive


In [85]:
!pip install voila
!jupyter serverextension enable voila —sys-prefix

Enabling: voila
- Writing config: /root/.jupyter
    - Validating...
Error loading server extension voila
      X is voila importable?
Enabling: —sys-prefix
- Writing config: /root/.jupyter
    - Validating...
Error loading server extension —sys-prefix
      X is —sys-prefix importable?


In [2]:
from fastbook import *
from fastai.vision.widgets import *
key = os.environ.get('AZURE_SEARCH_KEY', '517e9869251c41a0acdcb8446fd1c8f0')

In [11]:
PAC_PATH = Path('gdrive/My Drive/PrimatesAndCyborgs')
classes = tuple([fn for fn in os.listdir(PAC_PATH) if fn!='models'])
print('classes:\t', classes)
models_path = PAC_PATH/'models'
print('models_path:\t', models_path)

classes:	 ('homo sapiens', 'cyborg', 'chimpanzee', 'gorilla', 'orangutan', 'siamang', 'ring-tailed lemur', 'aye-aye', 'sifaka')
models_path:	 gdrive/My Drive/PrimatesAndCyborgs/models


In [96]:
pac_model = load_learner(models_path/'pac_model.pkl')
pac_model_vocab = pac_model.dls.vocab
print(pac_model_vocab)

button_upload = widgets.FileUpload()
widget_image = widgets.Output()
#widget_result = widgets.Label()
widget_result = widgets.HTML()
button_run = widgets.Button(description='Classify')

def button_action(change):
  image = PILImage.create(button_upload.data[-1])
  widget_image.clear_output()
  with widget_image: display(image.to_thumb(256,256))
  pred, pred_idx, probas = pac_model.predict(image)
  
  results_df = pd.Series(probas).round(decimals=4).to_frame(name='Probability')
  results_df.index = pac_model_vocab
  results_df.sort_values(by='Probability', ascending=False, inplace=True)
  results_df.reset_index(drop=False, inplace=True)
  results_df.columns = ['Species', 'Probability']
  
  widget_result.value =  results_df.to_html(index=False)#''.join([f'<p>{class_}:\t {probas[i]:.04f}</p>' for i, class_ in enumerate(pac_model_vocab)])
  
button_run.on_click(button_action)

['aye-aye', 'chimpanzee', 'cyborg', 'gorilla', 'homo sapiens', 'orangutan', 'ring-tailed lemur', 'siamang', 'sifaka']


In [97]:
box = VBox([
            widgets.HTML(value='<h2>Upload an image</h2>'),
            button_upload,
            button_run,
            widget_image,
            widget_result
])
box

In [63]:
??torch.round